In [531]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [532]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
#file_beg = '../NHANES-Downloader/data/csv_data/'

file_beg = '../NHANES-Downloader/data/csv_data/'

In [533]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')
files10 = glob.glob(file_beg+'2017-2018/*/*.csv')

In [534]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9, files10]

In [535]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [536]:
demo_indx = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [537]:
dfs = defaultdict(int)
for i in range(0,len(demo_indx)):
    dfs[i] = pd.read_csv(file_list_dict[i][demo_indx[i]])

## Important Variables

In [538]:
#EDIT: Removed INDHHINC from each after DMDHHSIZ and before INDFMINC
#EDIT: Removed INDHHIN2 from each after DMDHHSIZ and before INDFMIN2
#EDIT: Removed DMDAGE before DMDREDU before WT variable

#1999-2002
var_names = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN", "DMDCITZN", "DMDHHSIZ", "INDFMINC", "DMDHREDU", "WTINT4YR", "WTMEC4YR", "RIDSTATR", "SDMVPSU", "SDMVSTRA"]

#2003-2006
#RIDRETH1 is the only race/ethnicity variable in the 2005-2006 Demographics data file. The Demographics files that were released for NHANES 1999-2004 had a variable called ‘RIDRETH2’ that provided an analytic link to the NHANES III race/ethnicity variable.
var_names1 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN", "DMDCITZN", "DMDHHSIZ", "INDFMINC", "DMDHREDU", "WTINT2YR", "WTMEC2YR", "RIDSTATR", "SDMVPSU", "SDMVSTRA"]

#2007-2010
#DMDHRBRN becomes DMDBORN2
#Recategorize DMDHBRN
#INDFMINC becomes INDFMIN2 - Recategorize 14 and 15 to 11 to match 1999-2006
#INDHHINC becomes INDHHIN2 - Recategorize 14 and 15 to 11 to match 1999-2006
var_names2 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN2", "DMDCITZN", "DMDHHSIZ", "INDFMIN2", "DMDHREDU", "WTINT2YR", "WTMEC2YR", "RIDSTATR", "SDMVPSU", "SDMVSTRA"]

#2011-2016
#DMDHRBRN4 becomes 2 categories
#Recategorize all previous
var_names3 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN4", "DMDCITZN", "DMDHHSIZ", "INDFMIN2", "DMDHREDU", "WTINT2YR", "WTMEC2YR", "RIDSTATR", "SDMVPSU", "SDMVSTRA"]

#2017-2018
#DMDHRBRN4 becomes 2 categories
#DMDHREDU becomes DMDHREDZ
#Recategorize all previous
var_names4 = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN4", "DMDCITZN", "DMDHHSIZ", "INDFMIN2", "DMDHREDZ", "WTINT2YR", "WTMEC2YR", "RIDSTATR", "SDMVPSU", "SDMVSTRA"]

descr = ['SEQN', 'Gender', 'Age at Screening (Years)', 'Race/Hispanic Origin', 'Country of Birth', 'Citizenship Status','Total Number of People in the Household','Annual Family Income','Education Level','Two Year Int. Weight','Two Year MEC Weight','Loc. of Interview','Masked Variance','Masked Variance Pseudo-Stratum']

In [539]:
#To display all columns in Jupyter Notebooks
pd.set_option('display.max_columns', 500)

## Functions: Recategorize values, Count Values, Drop Rows

In [540]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Make a copy of the dataframes

In [541]:
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names1].copy() #2003-2004
df3 = dfs[3][var_names1].copy() #2005-2006
df4 = dfs[4][var_names2].copy() #2007-2008
df5 = dfs[5][var_names2].copy() #2009-2010
df6 = dfs[6][var_names3].copy() #2011-2012
df7 = dfs[7][var_names3].copy() #2013-2014
df8 = dfs[8][var_names3].copy() #2015-2016
df9 = dfs[9][var_names4].copy() #2017-2018

In [542]:
df0.shape

(9965, 14)

In [543]:
df8.shape

(9971, 14)

# Take data of only RIDSTATR = 2; Where individual was both interviewed and MEC examined

In [544]:
df0 = df0[df0['RIDSTATR']==2] #1999-2000
df1 = df1[df1['RIDSTATR']==2] #2001-2002
df2 = df2[df2['RIDSTATR']==2] #2003-2004
df3 = df3[df3['RIDSTATR']==2] #2005-2006
df4 = df4[df4['RIDSTATR']==2] #2007-2008
df5 = df5[df5['RIDSTATR']==2] #2009-2010
df6 = df6[df6['RIDSTATR']==2] #2011-2012
df7 = df7[df7['RIDSTATR']==2] #2013-2014
df8 = df8[df8['RIDSTATR']==2] #2015-2016

## Weight for 18-years: 1999 - 2018

In [545]:
#Create column for weight
def reweight(df, col_name, year_wt, weight):
    df[col_name] = df[year_wt]*weight

In [546]:
reweight(df0, 'MEC18YR', 'WTMEC4YR', 2/9)
reweight(df1, 'MEC18YR', 'WTMEC4YR', 2/9)
reweight(df2, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df3, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df4, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df5, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df6, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df7, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df8, 'MEC18YR', 'WTMEC2YR', 1/9)
reweight(df9, 'MEC18YR', 'WTMEC2YR', 1/9)

## Remove 2-yr and 4-yr weights

In [547]:
#Drop columns
def drop_cols(df, cols):
    df.drop(cols, axis=1, inplace=True)

In [548]:
drop_cols(df0,['WTINT4YR','WTMEC4YR'])
drop_cols(df1,['WTINT4YR','WTMEC4YR'])

In [549]:
drop_cols(df2,['WTINT2YR','WTMEC2YR'])
drop_cols(df3,['WTINT2YR','WTMEC2YR'])
drop_cols(df4,['WTINT2YR','WTMEC2YR'])
drop_cols(df5,['WTINT2YR','WTMEC2YR'])
drop_cols(df6,['WTINT2YR','WTMEC2YR'])
drop_cols(df7,['WTINT2YR','WTMEC2YR'])
drop_cols(df8,['WTINT2YR','WTMEC2YR'])
drop_cols(df9,['WTINT2YR','WTMEC2YR'])

In [550]:
df0[:5]

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,DMDBORN,DMDCITZN,DMDHHSIZ,INDFMINC,DMDHREDU,RIDSTATR,SDMVPSU,SDMVSTRA,MEC18YR
0,1.0,2.0,2.0,4.0,1.0,1.0,3.0,3.0,3.0,2.0,1.0,5.0,990.268132
1,2.0,1.0,77.0,3.0,1.0,1.0,1.0,8.0,5.0,2.0,3.0,1.0,3408.044382
2,3.0,2.0,10.0,3.0,3.0,2.0,4.0,6.0,4.0,2.0,2.0,7.0,4724.103694
3,4.0,1.0,1.0,4.0,1.0,1.0,7.0,3.0,3.0,2.0,1.0,2.0,1013.864237
4,5.0,1.0,49.0,3.0,1.0,1.0,3.0,11.0,4.0,2.0,2.0,8.0,10219.103963


## Recategorize values

In [551]:
#Recategorize DMDBORN to 2 categories
recategorize(df0, 'DMDBORN', {3:2, 7:77, 9:99}) #1999-2000
recategorize(df1, 'DMDBORN', {3:2, 7:77, 9:99}) #2001-2002
recategorize(df2, 'DMDBORN', {3:2, 7:77, 9:99}) #2003-2004
recategorize(df3, 'DMDBORN', {3:2, 7:77, 9:99}) #2005-2006


#Recategorize DMDBORN2 to 2 categories
#Recategorize INHHIN2 to INDHHINC
#Recategorize INDFMIN2 to INDFMINC

#2007-2008
recategorize(df4, 'DMDBORN2', {4:2, 5:2, 7:77, 9:99})
#recategorize(df4, 'INDHHIN2', {14:11, 15:11})
recategorize(df4, 'INDFMIN2', {14:11, 15:11})

#2009-2010
recategorize(df5, 'DMDBORN2', {4:2, 5:2, 7:77, 9:99})
#recategorize(df5, 'INDHHIN2', {14:11, 15:11})
recategorize(df5, 'INDFMIN2', {14:11, 15:11})

#2011-2012
#recategorize(df6, 'INDHHIN2', {14:11, 15:11})
recategorize(df6, 'INDFMIN2', {14:11, 15:11})

#2013-2014
#recategorize(df7, 'INDHHIN2', {14:11, 15:11})
recategorize(df7, 'INDFMIN2', {14:11, 15:11})

#2015-2016
#recategorize(df8, 'INDHHIN2', {14:11, 15:11})
recategorize(df8, 'INDFMIN2', {14:11, 15:11})

#2017-2018
#recategorize(df9, 'INDHHIN2', {14:11, 15:11})
recategorize(df9, 'INDFMIN2', {14:11, 15:11})

## Rename columns 1999 - 2018

In [552]:
df0

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,DMDBORN,DMDCITZN,DMDHHSIZ,INDFMINC,DMDHREDU,RIDSTATR,SDMVPSU,SDMVSTRA,MEC18YR
0,1.0,2.0,2.0,4.0,1.0,1.0,3.0,3.0,3.0,2.0,1.0,5.0,990.268132
1,2.0,1.0,77.0,3.0,1.0,1.0,1.0,8.0,5.0,2.0,3.0,1.0,3408.044382
2,3.0,2.0,10.0,3.0,2.0,2.0,4.0,6.0,4.0,2.0,2.0,7.0,4724.103694
3,4.0,1.0,1.0,4.0,1.0,1.0,7.0,3.0,3.0,2.0,1.0,2.0,1013.864237
4,5.0,1.0,49.0,3.0,1.0,1.0,3.0,11.0,4.0,2.0,2.0,8.0,10219.103963
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9959,9960.0,1.0,85.0,1.0,2.0,1.0,3.0,2.0,1.0,2.0,1.0,4.0,333.674902
9960,9961.0,1.0,36.0,2.0,2.0,2.0,4.0,7.0,1.0,2.0,2.0,7.0,8134.398689
9961,9962.0,1.0,4.0,3.0,1.0,1.0,4.0,9.0,5.0,2.0,2.0,1.0,5331.970521
9963,9964.0,1.0,11.0,3.0,1.0,1.0,3.0,2.0,4.0,2.0,2.0,3.0,5923.058204


In [553]:
#New column names
col_names = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH1", "DMDBORN4", "DMDCITZN", "DMDHHSIZ", "INDFMINC", "DMDHREDU", "RIDSTATR", "SDMVPSU", "SDMVSTRA", "MEC18YR"]

In [554]:
df0.columns = col_names
df1.columns = col_names
df2.columns = col_names
df3.columns = col_names
df4.columns = col_names
df5.columns = col_names
df6.columns = col_names
df7.columns = col_names
df8.columns = col_names
df9.columns = col_names

## Append years 1999 - 2018

In [555]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016","2017-2018"]

In [556]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]

In [557]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [558]:
result_1999_2018 = pd.concat(frames, keys = years)

In [559]:
result_1999_2018_cleaned = result_1999_2018.copy()

In [560]:
#Getting rows by keys
#result_1999_2016.loc[years[0]][:5]

In [561]:
len(result_1999_2018)

97316

## Count values for each column

In [562]:
len(col_names)

13

In [563]:
#RIAGENDR - Gender of the sample person
count_vals(result_1999_2018, col_names[1])

RIAGENDR
1.0    47996
2.0    49320
Name: SEQN, dtype: int64 

 NaN:  0


In [564]:
#RIDAGEYR - Age at Screening Adjudicated
count_vals(result_1999_2018, col_names[2])

RIDAGEYR
5.397605e-79    4354
1.000000e+00    2839
2.000000e+00    2908
3.000000e+00    1980
4.000000e+00    2087
                ... 
8.100000e+01     234
8.200000e+01     182
8.300000e+01     141
8.400000e+01     152
8.500000e+01     616
Name: SEQN, Length: 86, dtype: int64 

 NaN:  0


In [565]:
#RIDRETH1 - Race/Ethnicity
count_vals(result_1999_2018, col_names[3])

RIDRETH1
1.0    21637
2.0     7927
3.0    35791
4.0    22868
5.0     9093
Name: SEQN, dtype: int64 

 NaN:  0


In [566]:
#DMDBORN4 - Country of Birth
count_vals(result_1999_2018, col_names[4])

DMDBORN4
1.0     79608
2.0     17670
77.0       23
99.0        8
Name: SEQN, dtype: int64 

 NaN:  7


In [567]:
#DMDCITZN - Citizenship status
count_vals(result_1999_2018, col_names[5])

DMDCITZN
1.0    86950
2.0    10174
7.0      113
9.0       52
Name: SEQN, dtype: int64 

 NaN:  27


In [568]:
#DMDHHSIZ - Total number of people in the Household
count_vals(result_1999_2018, col_names[6])

DMDHHSIZ
1.0     7604
2.0    18276
3.0    17066
4.0    20418
5.0    15830
6.0     8583
7.0     9539
Name: SEQN, dtype: int64 

 NaN:  0


In [569]:
#INDHHINC - Annual Household Income
count_vals(result_1999_2018, col_names[7])

INDFMINC
1.0      4094
2.0      5413
3.0      8046
4.0      7482
5.0      8050
6.0     11391
7.0      8761
8.0      7064
9.0      5167
10.0     4149
11.0    19270
12.0     2397
13.0     1429
77.0     1682
99.0     1421
Name: SEQN, dtype: int64 

 NaN:  1500


In [570]:
#INDFMINC - Annual Family Income
count_vals(result_1999_2018, col_names[7])

INDFMINC
1.0      4094
2.0      5413
3.0      8046
4.0      7482
5.0      8050
6.0     11391
7.0      8761
8.0      7064
9.0      5167
10.0     4149
11.0    19270
12.0     2397
13.0     1429
77.0     1682
99.0     1421
Name: SEQN, dtype: int64 

 NaN:  1500


In [571]:
#DMDHRAGE - HH Ref Person Age
count_vals(result_1999_2018, col_names[9])

RIDSTATR
1.0      550
2.0    96766
Name: SEQN, dtype: int64 

 NaN:  0


In [572]:
#DMDHREDU - HH reference person's education level
count_vals(result_1999_2018, col_names[8])

DMDHREDU
1.0    12248
2.0    19586
3.0    22071
4.0    23006
5.0    16883
7.0       47
9.0      350
Name: SEQN, dtype: int64 

 NaN:  3125


In [573]:
len(result_1999_2018)

97316

In [574]:
result_1999_2018_cleaned[:5]

SEQN  RIAGENDR  RIDAGEYR  RIDRETH1  DMDBORN4  DMDCITZN  DMDHHSIZ  \
1999-2000 0   1.0       2.0       2.0       4.0       1.0       1.0       3.0   
          1   2.0       1.0      77.0       3.0       1.0       1.0       1.0   
          2   3.0       2.0      10.0       3.0       2.0       2.0       4.0   
          3   4.0       1.0       1.0       4.0       1.0       1.0       7.0   
          4   5.0       1.0      49.0       3.0       1.0       1.0       3.0   

             INDFMINC  DMDHREDU  RIDSTATR  SDMVPSU  SDMVSTRA       MEC18YR  \
1999-2000 0       3.0       3.0       2.0      1.0       5.0    990.268132   
          1       8.0       5.0       2.0      3.0       1.0   3408.044382   
          2       6.0       4.0       2.0      2.0       7.0   4724.103694   
          3       3.0       3.0       2.0      1.0       2.0   1013.864237   
          4      11.0       4.0       2.0      2.0       8.0  10219.103963   

                  Year  
1999-2000 0  1999-2000  
          1  1999-2000  
          2  1999-2000  
          3  1999-2000  
          4  1999-2000

## Remove rows with missing values:

In [575]:
drop_rows(result_1999_2018_cleaned, col_names[1], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[2], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[3], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[4], [np.nan, 77, 99])
drop_rows(result_1999_2018_cleaned, col_names[5], [np.nan, 7, 9])
drop_rows(result_1999_2018_cleaned, col_names[6], [np.nan])
drop_rows(result_1999_2018_cleaned, col_names[7], [np.nan, 12, 13, 77, 99])
drop_rows(result_1999_2018_cleaned, col_names[8], [np.nan, 7, 9]) 

##FIX
drop_rows(result_1999_2018_cleaned, col_names[9], [np.nan]) 

In [576]:
len(result_1999_2018_cleaned)

86320

In [577]:
result_1999_2018_cleaned[:5]

SEQN  RIAGENDR  RIDAGEYR  RIDRETH1  DMDBORN4  DMDCITZN  DMDHHSIZ  \
1999-2000 0   1.0       2.0       2.0       4.0       1.0       1.0       3.0   
          1   2.0       1.0      77.0       3.0       1.0       1.0       1.0   
          2   3.0       2.0      10.0       3.0       2.0       2.0       4.0   
          3   4.0       1.0       1.0       4.0       1.0       1.0       7.0   
          4   5.0       1.0      49.0       3.0       1.0       1.0       3.0   

             INDFMINC  DMDHREDU  RIDSTATR  SDMVPSU  SDMVSTRA       MEC18YR  \
1999-2000 0       3.0       3.0       2.0      1.0       5.0    990.268132   
          1       8.0       5.0       2.0      3.0       1.0   3408.044382   
          2       6.0       4.0       2.0      2.0       7.0   4724.103694   
          3       3.0       3.0       2.0      1.0       2.0   1013.864237   
          4      11.0       4.0       2.0      2.0       8.0  10219.103963   

                  Year  
1999-2000 0  1999-2000  
          1  1999-2000  
          2  1999-2000  
          3  1999-2000  
          4  1999-2000

## See if missing values have been correctly removed:

In [578]:
count_vals(result_1999_2018_cleaned, col_names[1])

RIAGENDR
1.0    42729
2.0    43591
Name: SEQN, dtype: int64 

 NaN:  0


In [579]:
count_vals(result_1999_2018_cleaned, col_names[2])

RIDAGEYR
5.397605e-79    3929
1.000000e+00    2558
2.000000e+00    2609
3.000000e+00    1774
4.000000e+00    1872
                ... 
8.100000e+01     204
8.200000e+01     164
8.300000e+01     119
8.400000e+01     128
8.500000e+01     530
Name: SEQN, Length: 86, dtype: int64 

 NaN:  0


In [580]:
count_vals(result_1999_2018_cleaned, col_names[3])

RIDRETH1
1.0    18565
2.0     6683
3.0    32947
4.0    20247
5.0     7878
Name: SEQN, dtype: int64 

 NaN:  0


In [581]:
count_vals(result_1999_2018_cleaned, col_names[4])

DMDBORN4
1.0    71596
2.0    14724
Name: SEQN, dtype: int64 

 NaN:  0


In [582]:
count_vals(result_1999_2018_cleaned, col_names[5])

DMDCITZN
1.0    77914
2.0     8406
Name: SEQN, dtype: int64 

 NaN:  0


In [583]:
count_vals(result_1999_2018_cleaned, col_names[6])

DMDHHSIZ
1.0     7047
2.0    16448
3.0    15337
4.0    18232
5.0    14078
6.0     7462
7.0     7716
Name: SEQN, dtype: int64 

 NaN:  0


In [584]:
count_vals(result_1999_2018_cleaned, col_names[7])

INDFMINC
1.0      3874
2.0      5240
3.0      7806
4.0      7249
5.0      7798
6.0     11137
7.0      8560
8.0      6852
9.0      5061
10.0     4053
11.0    18690
Name: SEQN, dtype: int64 

 NaN:  0


In [585]:
count_vals(result_1999_2018_cleaned, col_names[8])

DMDHREDU
1.0    10683
2.0    17783
3.0    20303
4.0    21593
5.0    15958
Name: SEQN, dtype: int64 

 NaN:  0


In [586]:
before = len(result_1999_2018)
before

97316

In [587]:
after = len(result_1999_2018_cleaned)
after

86320

In [588]:
(before-after)/after

0.127386468952734

In [589]:
result_1999_2018_cleaned.isnull().any()

SEQN        False
RIAGENDR    False
RIDAGEYR    False
RIDRETH1    False
DMDBORN4    False
DMDCITZN    False
DMDHHSIZ    False
INDFMINC    False
DMDHREDU    False
RIDSTATR    False
SDMVPSU     False
SDMVSTRA    False
MEC18YR     False
Year        False
dtype: bool

## MongoDB Insertion

In [590]:
#Import MongoClient
from pymongo import MongoClient

In [591]:
#Create a MongoClient to run the MongoDB instance
client = MongoClient("localhost", 27017)

In [592]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [593]:
#Creating a database
db = client['NHANES']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NHANES')

In [594]:
db.list_collection_names()

['alq', 'demo_p', 'bpq', 'bmx', 'bpx']

In [595]:
#Creating a collection
demo_p = db.demo_p

In [596]:
#If collections exist, then drop
if 'demo_p' in db.list_collection_names():
    demo_p.drop()
    db.list_collection_names()

## Inputting into DB

In [597]:
result_1999_2018_cleaned.rename(columns= {'SEQN':'_id'}, inplace=True)

In [598]:
result_1999_2018_cleaned[:3]

_id  RIAGENDR  RIDAGEYR  RIDRETH1  DMDBORN4  DMDCITZN  DMDHHSIZ  \
1999-2000 0  1.0       2.0       2.0       4.0       1.0       1.0       3.0   
          1  2.0       1.0      77.0       3.0       1.0       1.0       1.0   
          2  3.0       2.0      10.0       3.0       2.0       2.0       4.0   

             INDFMINC  DMDHREDU  RIDSTATR  SDMVPSU  SDMVSTRA      MEC18YR  \
1999-2000 0       3.0       3.0       2.0      1.0       5.0   990.268132   
          1       8.0       5.0       2.0      3.0       1.0  3408.044382   
          2       6.0       4.0       2.0      2.0       7.0  4724.103694   

                  Year  
1999-2000 0  1999-2000  
          1  1999-2000  
          2  1999-2000

In [599]:
demo_p_dict = result_1999_2018_cleaned.to_dict(orient='records')

In [600]:
demo_p_dict[0]

{'_id': 1.0,
 'RIAGENDR': 2.0,
 'RIDAGEYR': 2.0,
 'RIDRETH1': 4.0,
 'DMDBORN4': 1.0,
 'DMDCITZN': 1.0,
 'DMDHHSIZ': 3.0,
 'INDFMINC': 3.0,
 'DMDHREDU': 3.0,
 'RIDSTATR': 2.0,
 'SDMVPSU': 1.0,
 'SDMVSTRA': 5.0,
 'MEC18YR': 990.268132,
 'Year': '1999-2000'}

In [601]:
#Insert collection
demo_p.insert_many(demo_p_dict)

In [602]:
db.list_collection_names()

['alq', 'bpq', 'bmx', 'bpx', 'demo_p']

## Dataframe to CSV

In [603]:
result_1999_2018_cleaned.to_csv('DEMO.csv')